# Repairing Code

_Brief abstract/introduction/motivation.  State what the chapter is about in 1-2 paragraphs._
_Then, have an introduction video:_

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Fuzzer.ipynb).

In [ ]:
import bookutils

## Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## Fixing Things Manually

\todo{Add}

## Automatic Code Repair

1. Have a set of tests.
2. Localize the defect.
3. Apply random insert/delete/swap operations.
4. Evolve.

Let's do this on `middle` first.

## A Test Suite

A set of passing tests.

In [ ]:
from StatisticalDebugger import middle

In [ ]:
import random

In [ ]:
def middle_testcase():
    x = random.randrange(10)
    y = random.randrange(10)
    z = random.randrange(10)
    return x, y, z

In [ ]:
[middle_testcase() for i in range(5)]

In [ ]:
def middle_test(x, y, z):
    m = middle(x, y, z)
    assert m == sorted([x, y, z])[1]

In [ ]:
def middle_passing_testcase():
    while True:
        try:
            x, y, z = middle_testcase()
            _ = middle_test(x, y, z)
            return x, y, z
        except AssertionError:
            pass

In [ ]:
(x, y, z) = middle_passing_testcase()
m = middle(x, y, z)
print(f"middle({x}, {y}, {z}) = {m}")

In [ ]:
def middle_failing_testcase():
    while True:
        try:
            x, y, z = middle_testcase()
            _ = middle_test(x, y, z)
        except AssertionError:
            return x, y, z

In [ ]:
(x, y, z) = middle_failing_testcase()
m = middle(x, y, z)
print(f"middle({x}, {y}, {z}) = {m}")

In [ ]:
MIDDLE_TESTS = 100

In [ ]:
MIDDLE_PASSING_TESTCASES = [middle_passing_testcase() for i in range(MIDDLE_TESTS)]

In [ ]:
MIDDLE_FAILING_TESTCASES = [middle_failing_testcase() for i in range(MIDDLE_TESTS)]

## Locating the Defect

In [ ]:
from StatisticalDebugger import OchiaiDebugger, TarantulaDebugger, CoverageCollector

In [ ]:
debugger = OchiaiDebugger(CoverageCollector)

for x, y, z in MIDDLE_PASSING_TESTCASES:
    with debugger.collect_pass():
        m = middle(x, y, z)

for x, y, z in MIDDLE_FAILING_TESTCASES:
    with debugger.collect_fail():
        m = middle(x, y, z)

In [ ]:
debugger

In [ ]:
debugger.rank()

We thus focus on the following line:

In [ ]:
# ignore
import inspect
from bookutils import print_content

In [ ]:
# ignore
lineno = debugger.rank()[0]
lines, first_lineno = inspect.getsourcelines(middle)
print(lineno, end="")
print_content(lines[lineno - first_lineno], '.py')

In [ ]:
debugger.suspiciousness(lineno)

In [ ]:
debugger.suspiciousness(first_lineno)

## Random Code Mutations

* delete a statement
* insert a statement (from the same source)
* replace by another statements (from the same source)

In [ ]:
import ast
import astor

In [ ]:
from bookutils import rich_output

In [ ]:
if rich_output():
    from showast import show_ast
else:
    def show_ast(tree):
        ast.dump(tree)

In [ ]:
from ast import NodeTransformer, NodeVisitor

In [ ]:
def middle_tree():
    return ast.parse(inspect.getsource(middle))
show_ast(middle_tree())

### Picking Statements

Let us start with a source of potential statements.

In [ ]:
class StatementVisitor(NodeVisitor):
    def __init__(self):
        self.statements = []
        super().__init__()
        
    def add_statements(self, node, attr):
        elem = getattr(node, attr, [])
        if isinstance(elem, list):
            self.statements += elem
        else:
            self.statements.append(elem)
        
    def visit_Node(self, node):
        # Any node other than the ones listed below
        self.add_statements(node, 'body')
        self.add_statements(node, 'orelse')
            
    def visit_Module(self, node):
        # Module children are defs, classes and globals - don't add
        super().generic_visit(node)        

    def visit_ClassDef(self, node):
        # Class children are defs and globals - don't add
        super().generic_visit(node)        

    def generic_visit(self, node):
        self.visit_Node(node)
        super().generic_visit(node)

In [ ]:
def all_statements(tree, tp=None):
    visitor = StatementVisitor()
    visitor.visit(tree)
    statements = visitor.statements
    if tp is not None:
        statements = [s for s in statements if isinstance(s, tp)]

    return statements

In [ ]:
all_statements(middle_tree(), ast.Return)

We can randomly pick an element:

In [ ]:
import random

In [ ]:
random_node = random.choice(all_statements(middle_tree()))
astor.to_source(random_node)

### Mutating Statements

In [ ]:
import copy

In [ ]:
class RandomNodeMutator(NodeTransformer):
    def __init__(self, suspiciousness_func=None, 
                 statements=None, mutation_rate=0.1, log=False):
        
        if suspiciousness_func is None:
            suspiciousness_func = lambda lineno: 1.0
        self.suspiciousness_func = suspiciousness_func

        if statements is None:
            statements = []
        self.statements = copy.deepcopy(statements)

        self.mutation_rate = mutation_rate
        self.log = log

        super().__init__()

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    def node_suspiciousness(self, node):
        if not hasattr(node, 'lineno'):
            return 0
        return self.suspiciousness_func(node.lineno)

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    SKIP_LIST = {ast.Module, ast.ClassDef,
                 ast.FunctionDef, ast.AsyncFunctionDef}

    def should_mutate(self, node):
        if not isinstance(node, ast.stmt):
            return False
        for cls in self.SKIP_LIST:
            if isinstance(node, cls):
                return False

        suspicious_enough = (random.random() <= self.node_suspiciousness(node))
        chosen_for_mutation = (random.random() <= self.mutation_rate)
        return suspicious_enough and chosen_for_mutation

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    def format_node(self, node):
        if node is None:
            return None
        if isinstance(node, list):
            return "; ".join(self.format_node(elem) for elem in node)

        s = astor.to_source(node).replace('\n', ' ').strip()
        if len(s) > 10:
            s = s[:10] + "..."
        return s

    def generic_visit(self, node):
        if not self.should_mutate(node):
            return super().generic_visit(node)
        
        op = random.choice([self.insert, self.swap, self.delete])
        new_node = op(node)

        if self.log:
            print(f"{op.__name__}: {self.format_node(node)} becomes {self.format_node(new_node)}")

        return new_node

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    def choose_statement(self):
        return random.choice(self.statements)

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    def swap(self, node):
        # Replace with a random node from statements
        return self.choose_statement()

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    def insert(self, node):
        # Insert a random node from statements
        return [
            node,
            self.choose_statement()
        ]

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    def delete(self, node):
        # Delete this node
        return None

In [ ]:
class RandomNodeMutator(RandomNodeMutator):
    def visit(self, node):
        if not self.statements:
            self.statements = copy.deepcopy(all_statements(node))

        return super().visit(node)

In [ ]:
mutator = RandomNodeMutator(log=True)

In [ ]:
new_tree = mutator.visit(middle_tree())

In [ ]:
print_content(astor.to_source(new_tree), '.py')

## Synopsis

We can repair things!

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_